In [ ]:
"""
"""

In [ ]:
"""
categorical only + various estimator
numerical only + various estimator
variable selection (xgb, RF) + various estimator
numerical as categorical
second order features
standardize kfold transformer
numerical as categorical + 2nd order + linear
"""

In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import xgboost as xgb
import vw_utils as vw
import gini
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [100]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

hist = train.Hazard.value_counts()
toDown = hist[hist.cumsum() / train.shape[0] < 0.8].index.values
cap = train.Hazard[~train.Hazard.isin(toDown)].quantile(0.95)

train['Hazard_cap'] = np.minimum(train.Hazard, cap)

data = pd.concat([train, test], axis=0).reset_index()
train_set = np.hstack([np.ones(train.shape[0]), np.zeros(test.shape[0])])

## --------------------- Feature Start -----------------------

In [97]:
# start
categorical = ['T1_V' + str(i) for i in list(range(4, 10)) + [11, 12, 15, 16, 17]] + \
                ['T2_V' + str(i) for i in [3, 5, 11, 12, 13]]

numerical = set(train.columns).difference(categorical + ['Hazard', 'Hazard_cap', 'Id'])
numerical = list(numerical)

In [98]:
# raw feature
from sklearn.preprocessing import StandardScaler
import re

SS = StandardScaler()
feat_raw = data[numerical].applymap(float)
feat_raw[numerical] = SS.fit_transform(feat_raw[numerical])


toDrop = re.sub('[ ]+', '_', data[categorical].ix[0, :].to_string()).split('\n')
feat_raw_cat = pd.get_dummies(data[categorical]).drop(toDrop, axis=1)

feat_raw = pd.concat([feat_raw, feat_raw_cat], axis=1)
feat_raw.shape

(101999, 95)

In [101]:
# assemble features

tmp = pd.concat([feat_raw], axis=1)
train_c = tmp[train_set==1]
test_c = tmp[train_set==0]

print(train_c.shape)
print(test_c.shape)

(50999, 95)
(51000, 95)


## --------------------- Modeling -----------------------

In [107]:
%%time
# rank min

import gini
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import KFold

param = {'max_depth':7, 'eta':0.01, 'objective':'reg:linear',
         'gamma':0, 'min_child_weight': 30, 'subsample': 0.8,
         'colsample_bytree': 0.5}

scores = []
for idx_train, idx_test in KFold(train.shape[0], 10):
    print('yes')
    dTrain = xgb.DMatrix(train_c.ix[idx_train], label=train.Hazard_cap[idx_train])
    dTest = xgb.DMatrix(train_c.ix[idx_test])
    model = xgb.train(param, dTrain, 1000)
    pred = model.predict(dTest)
    scores.append(gini.normalized_gini(train.Hazard[idx_test], pred))




yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
CPU times: user 8min 23s, sys: 0 ns, total: 8min 23s
Wall time: 2min 11s


In [51]:
import gini

important = (pd.Series(model.get_fscore())            
             .sort(inplace=False, ascending=False)
             .reset_index()['index']
             .ix[:70]
             .map(lambda x: int(x[1:]))
             .values)

dTrain = xgb.DMatrix(train_c.ix[:, important], label=train.Hazard)
result = xgb.cv(param, dTrain, 2000, feval=gini.xgb_gini)

[0]	cv-test-gini:0.286834+0.014692	cv-train-gini:0.327518+0.014787
[1]	cv-test-gini:0.291687+0.005753	cv-train-gini:0.332285+0.007709
[2]	cv-test-gini:0.295558+0.007205	cv-train-gini:0.343513+0.001941
[3]	cv-test-gini:0.304520+0.005374	cv-train-gini:0.351494+0.003539
[4]	cv-test-gini:0.317261+0.008228	cv-train-gini:0.362146+0.001009
[5]	cv-test-gini:0.322478+0.009983	cv-train-gini:0.368219+0.002485
[6]	cv-test-gini:0.325479+0.009573	cv-train-gini:0.368166+0.001194
[7]	cv-test-gini:0.327013+0.009679	cv-train-gini:0.369551+0.001064
[8]	cv-test-gini:0.328550+0.010312	cv-train-gini:0.373067+0.002491
[9]	cv-test-gini:0.329454+0.009658	cv-train-gini:0.373385+0.001329
[10]	cv-test-gini:0.330436+0.009629	cv-train-gini:0.375032+0.001306
[11]	cv-test-gini:0.333072+0.008409	cv-train-gini:0.377119+0.001495
[12]	cv-test-gini:0.334517+0.006172	cv-train-gini:0.379361+0.001015
[13]	cv-test-gini:0.334835+0.007014	cv-train-gini:0.379708+0.001513
[14]	cv-test-gini:0.335227+0.007177	cv-train-gini:0.381922

In [44]:
pd.DataFrame({'Id': test.Id, 'Hazard': yhat}).reindex_axis(['Id', 'Hazard'], 1).to_csv('../output/ensemble_07.csv', index=0)